In [1]:
%pip install tensorflow

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel,TFBertForSequenceClassification
from tensorflow import keras
from transformers import AdamW
from sklearn.model_selection import train_test_split
import re
import string
from sklearn.preprocessing import LabelEncoder

Note: you may need to restart the kernel to use updated packages.


2024-12-10 15:33:11.119794: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 15:33:11.137124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 15:33:11.142520: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 15:33:11.155836: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/vlad.cristescu/miniconda3/env

In [2]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


2024-12-10 15:33:14.650909: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1001 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6
2024-12-10 15:33:14.653076: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43611 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:c1:00.0, compute capability: 8.6
2024-12-10 15:33:14.654882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43611 MB memory:  -> device: 2, name: NVIDIA A40, pci bus id: 0000:e1:00.0, compute capability: 8.6


Number of devices: 3


In [3]:
file_path = "../datasets/fake_or_real_news.csv"
data = pd.read_csv(file_path)
print(data["text"].head())

0    Daniel Greenfield, a Shillman Journalism Fello...
1    Google Pinterest Digg Linkedin Reddit Stumbleu...
2    U.S. Secretary of State John F. Kerry said Mon...
3    — Kaydee King (@KaydeeKing) November 9, 2016 T...
4    It's primary day in New York and front-runners...
Name: text, dtype: object


In [4]:
texts = data["text"].tolist()
labels = data["label"].tolist()

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [5]:
def preprocess_for_bert(text):
    text = text.lower()  # Transformă în lowercase
    text = re.sub(r"https?://\S+|www\.\S+", '', text)  # Elimină URL-urile
    text = re.sub(r"<.*?>+", '', text)  # Elimină HTML-ul
    text = re.sub(r'\n', ' ', text)  # Înlocuiește newline cu spațiu
    text = re.sub(r'\s+', ' ', text).strip()  # Elimină spațiile multiple
    return text

In [6]:
train_texts, test_texts, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

max_length = 128  # Poți ajusta dimensiunea maximă
batch_size = 32

train_encodings  = tokenizer.batch_encode_plus(
    train_texts,
    max_length=max_length,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts,
    max_length=max_length,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

train_inputs = {
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"]
}
test_inputs = {
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"]
}
train_labels = tf.convert_to_tensor(y_train)
test_labels = tf.convert_to_tensor(y_test)

/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
with strategy.scope():
    bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

    bert_model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    history = bert_model.fit(
        train_inputs,
        train_labels,
        validation_data=(test_inputs, test_labels),
        epochs=3,
        batch_size=batch_size
    )

    # Evaluarea modelului
    results = bert_model.evaluate(test_inputs, test_labels, batch_size=batch_size)
    print(f"Test Accuracy: {results[1] * 100:.2f}%")

    # Salvarea modelului și a tokenizer-ului
    bert_model.save_pretrained("salvat_model_bert")
    tokenizer.save_pretrained("salvat_tokenizer")

print(bert_model.summary())

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
INFO:tensorflow:Collective all_reduce tensors: 198 all_reduces, num_devices = 3, group_size = 3, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce IndexedSlices: 3 all_reduces, num_devices =3, group_size = 3, implementation = CommunicationImplementation.NCCL
INFO:tensorflow:Error reported to Coordinator: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[30522,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill] name: 
Traceback (most recent call last):
  File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tensorflow/python/training/coordinator.py", line 293, in stop_on_exception
    yield
  File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tensorflow/python/distribute/mirrored_run.py", line 387, in run
    self.main_result = self.main_fn(*se

2024-12-10 15:36:00.132858: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 89.42MiB (rounded to 93763584)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-12-10 15:36:00.132947: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-12-10 15:36:00.132970: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 17, Chunks in use: 17. 4.2KiB allocated for chunks. 4.2KiB in use in bin. 96B client-requested in use in bin.
2024-12-10 15:36:00.132984: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-12-10 15:36:00.133002: I external/local_tsl

ResourceExhaustedError: in user code:

    File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1709, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tf_keras/src/optimizers/optimizer.py", 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Num GPUs Available: 3


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7fef8a3fbbf0>

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Configurație GPU
strategy = tf.distribute.MirroredStrategy()  # Utilizează toate GPU-urile disponibile
print(f"Num GPUs Available: {strategy.num_replicas_in_sync}")

# 1. Configurare date de exemplu
texts = [
    "The weather is nice today.",
    "This is a fake news example.",
    "BERT is a powerful transformer model.",
    "This is another example of fake news."
]
labels = [0, 1, 0, 1]  # 0 pentru real, 1 pentru fake

# 2. Tokenizare
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_length = 128

def tokenize_texts(texts, tokenizer, max_length):
    tokens = tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return tokens["input_ids"], tokens["attention_mask"]

input_ids, attention_masks = tokenize_texts(texts, tokenizer, max_length)

# Împărțirea setului de date
train_ids, test_ids, train_masks, test_masks, train_labels, test_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42
)

# Conversia etichetelor în TensorFlow Tensors
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.float32)

# 3. Construirea modelului
with strategy.scope():
    bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)
    
    # Compilarea modelului
    bert_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

# 4. Antrenarea modelului
history = bert_model.fit(
    x={
        "input_ids": train_ids,
        "attention_mask": train_masks
    },
    y=train_labels,
    validation_data=(
        {
            "input_ids": test_ids,
            "attention_mask": test_masks
        },
        test_labels
    ),
    epochs=3,
    batch_size=8
)

# 5. Evaluarea modelului
results = bert_model.evaluate(
    x={
        "input_ids": test_ids,
        "attention_mask": test_masks
    },
    y=test_labels,
    batch_size=8
)
print(f"Test Accuracy: {results[1] * 100:.2f}%")

# 6. Salvarea modelului și tokenizer-ului
bert_model.save_pretrained("saved_bert_model")
tokenizer.save_pretrained("saved_bert_tokenizer")


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Num GPUs Available: 3


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([3, 0, 2])